# Bitcoin Turtle Trading Strategy Exploration

### Description

This project is an exploration of a very simplified version of the famous "Turtle"
trading strategy done on the Bitcoin price data.  The strategy consists of looking
at the past 4 weeks of price data and calculating the total range.  If price exceeds
the high of the range at any point on a given day a hypothetical long trade is
entered from a price level just above the range high.  We then calculate the difference
in this entry price and the closing price of Bitcoin 4 weeks later.  The analysis
is then done on these differences to determine if breaking out above a 4 week high
price leads to higher prices 4 weeks later.

### Data

The dataset is cryptocurrency daily price data from [Coinmarketcap.com](https://coinmarketcap.com/), aggregated at [Kaggle.com](https://www.kaggle.com/)

https://www.kaggle.com/philmohun/cryptocurrency-financial-data

## Import the data

The full dataset is made up of 200 cryptocurrencies and provides detailed price, volume, and market cap data for each over varying dates, depending on the coin itself.  The first 4 rows are all repeats of the column headers so we can omit.    

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
% matplotlib inline

In [7]:
# Read in all data and inspect
df_all = pd.read_csv('consolidated_coin_data.csv', skiprows=4, low_memory=False)
df_all.head()

,Currency,Date,Open,High,Low,Close,Volume,Market Cap
0,0x,"Feb 11, 2018",1.09,1.09,0.934155,0.979054,"4,888,770","555,363,000"
1,0x,"Feb 10, 2018",1.14,1.20,0.985705,1.10,"10,828,700","576,535,000"
2,0x,"Feb 09, 2018",1.08,1.15,1.01,1.14,"5,979,420","545,842,000"
3,0x,"Feb 08, 2018",0.988963,1.13,0.988963,1.07,"12,992,800","501,142,000"
4,0x,"Feb 07, 2018",1.01,1.16,0.902145,0.993355,"13,476,600","513,163,000"


In [9]:
df_all.dtypes

Currency      object
Date          object
Open          object
High          object
Low           object
Close         object
Volume        object
Market Cap    object
dtype: object

## Clean the data

A couple things to note here.  First of all, for our purposes we will only need to look at Bitcoin data, specifically the columns Currency, Data, High, Low and Close.  Also, we notice that the `dtype` for each column is `object` so we know well need to cast to `float` before performing computations on the data.

In [23]:
# Create a dataframe of only Bitcoin High, Low, Close
df_btc = df_all[df_all.Currency == 'bitcoin']
df_btc = df_btc[['Currency', 'Date', 'High', 'Low', 'Close']]

# Set make Data a datetime object and use as index
df_btc.Date = pd.to_datetime(df_btc.Date)
df_btc = df_btc.set_index('Date')

# Cast price data columns to floats
df_btc.High = df_btc.High.apply(pd.to_numeric)
df_btc.Low = df_btc.Low.apply(pd.to_numeric)
df_btc.Close = df_btc.Close.apply(pd.to_numeric)

df_btc.describe()

,High,Low,Close
count,1751.000000,1751.000000,1751.000000
mean,1491.354038,1376.062067,1441.005945
std,3033.574104,2723.897810,2899.302845
min,74.560000,65.530000,68.430000
25%,278.830000,267.865000,274.410000
50%,492.370000,468.790000,479.000000
75%,881.725000,824.280000,860.205000
max,20089.000000,18974.100000,19497.400000


In [24]:
df_btc.dtypes

Currency     object
High        float64
Low         float64
Close       float64
dtype: object